<a href="https://colab.research.google.com/github/Sakinat-Folorunso/CMP_805_Advanced_Programming_Languages/blob/main/notebooks/CMP805_Week7_PH_Python_Colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# CMP805 — Week 7 Practical (Python, Colab)
**Topic:** OO semantics — classes, interfaces, traits; Liskov Substitution Principle (LSP)
**Course:** Advanced Programming Languages (M.Sc.), OOU — CMP805

**Instructor:** **DR SAKINAT FOLORUNSO – ASSOCIATE PROFESSOR OF AI SYSTEMS AND FAIR DATA**  
**Department:** **COMPUTER SCIENCES, OLABISI ONABANJO UNIVERSITY, AGO‑IWOYE, OGUN STATE, NIGERIA**

> This PH implements a tiny OO core: **class table, dynamic dispatch, interface checking** and a classic **LSP pitfall & fix**.

### Learning goals (≈60 minutes)
- Implement **method lookup with dynamic dispatch** across a class hierarchy (single inheritance).
- Define **interfaces** as sets of method signatures and check **class satisfies interface**.
- Observe a classic **LSP violation** (Rectangle/Square) and refactor to a composition‑based design that satisfies substitutability.

In [ ]:
# 🧑‍🎓 Student info
STUDENT_NAME = "Type your full name here"
STUDENT_ID   = "Matric/ID here"
print("Student:", STUDENT_NAME, "| ID:", STUDENT_ID)

In [ ]:
# ✅ Environment check (Python 3.10+ for pattern matching)
import sys
major, minor = sys.version_info[:2]
assert (major, minor) >= (3, 10), f"Need Python 3.10+, found {major}.{minor}"
print(f"Python {major}.{minor} OK — match/case available.")

In [ ]:
# =====================================
# Part 1 — A tiny OO core: classes and objects
# =====================================
from __future__ import annotations
from dataclasses import dataclass
from typing import Dict, List, Tuple, Callable, Optional, Any

# --- Types for signatures (nominal) ---
@dataclass(frozen=True)
class TyInt: pass

@dataclass(frozen=True)
class TyBool: pass

@dataclass(frozen=True)
class TyClass:
    name: str  # nominal class type

Type = TyInt | TyBool | TyClass

def show_ty(t: Type) -> str:
    match t:
        case TyInt(): return "Int"
        case TyBool(): return "Bool"
        case TyClass(name): return name
        case _ : return str(t)

# --- Method signature ---
@dataclass(frozen=True)
class MethSig:
    name: str
    params: List[Tuple[str, Type]]   # (param name, type)
    ret: Type

# --- Method definition: body is a Python lambda we use as our "core calculus"
@dataclass(frozen=True)
class Method:
    sig: MethSig
    body: Callable[['Object', List['Value']], 'Value']  # body(this, args) -> Value

# --- Class definition ---
@dataclass
class ClassDef:
    name: str
    super: Optional[str]                      # single inheritance; None for root
    fields: List[Tuple[str, Type]]            # (field name, type)
    methods: Dict[str, Method]                # name -> method

# --- Runtime values ---
@dataclass
class ObjVal:
    cls: str
    fields: Dict[str, 'Value']

Value = int | bool | ObjVal

# --- Class table (global) ---
ClassTable = Dict[str, ClassDef]

In [ ]:
# =====================================
# Part 2 — Dynamic dispatch: method lookup (TO IMPLEMENT)
# =====================================
def lookup_method(CT: ClassTable, cls: str, mname: str) -> Method:
    """
    Find method mname starting from class 'cls' following 'super' links.
    Raise KeyError if not found.
    """
    # TODO: Implement dynamic method lookup across inheritance chain.
    # Hints:
    # 1) Look up the class in CT; 2) If method in that class, return it;
    # 3) Else follow 'super' if present; 4) Stop with KeyError if chain ends.
    cur = cls
    while True:
        if cur not in CT:
            raise KeyError(f"unknown class {cur}")
        cdef = CT[cur]
        if mname in cdef.methods:
            return cdef.methods[mname]
        if cdef.super is None:
            break
        cur = cdef.super
    raise KeyError(f"method {mname} not found in class {cls} or its superclasses")

In [ ]:
# =====================================
# Part 3 — Object creation and method call (uses lookup_method)
# =====================================
def new_object(CT: ClassTable, cname: str, *field_vals: Value) -> ObjVal:
    if cname not in CT: raise KeyError(f"unknown class {cname}")
    c = CT[cname]
    if len(field_vals) != len(c.fields):
        raise ValueError("field arity mismatch")
    fields = { nm: v for ( (nm,_), v ) in zip(c.fields, field_vals) }
    return ObjVal(cname, fields)

def call_method(CT: ClassTable, obj: ObjVal, mname: str, args: List[Value]) -> Value:
    m = lookup_method(CT, obj.cls, mname)         # dynamic dispatch via class of receiver
    return m.body(obj, args)                      # execute with 'this' = obj

In [ ]:
# =====================================
# Part 4 — Sample hierarchy: Point and ColoredPoint
# =====================================
Int = TyInt()
Bool = TyBool()

def mk_point_class() -> ClassDef:
    # class Point(x:Int, y:Int) { move(dx:Int, dy:Int): Point; is_origin(): Bool }
    def move_body(this: ObjVal, args: List[Value]) -> Value:
        dx, dy = args
        return ObjVal(this.cls, {"x": this.fields["x"] + dx, "y": this.fields["y"] + dy})
    def is_origin_body(this: ObjVal, args: List[Value]) -> Value:
        return (this.fields["x"] == 0) and (this.fields["y"] == 0)
    methods = {
        "move": Method(MethSig("move", [("dx", Int), ("dy", Int)], TyClass("Point")), move_body),
        "is_origin": Method(MethSig("is_origin", [], Bool), is_origin_body)
    }
    return ClassDef("Point", None, [("x", Int), ("y", Int)], methods)

def mk_colored_point_class() -> ClassDef:
    # class ColoredPoint(x:Int, y:Int, color:Int) extends Point {...}
    def set_color_body(this: ObjVal, args: List[Value]) -> Value:
        (c,) = args
        return ObjVal(this.cls, {"x": this.fields["x"], "y": this.fields["y"], "color": c})
    # Override move to preserve color
    def move_body(this: ObjVal, args: List[Value]) -> Value:
        dx, dy = args
        return ObjVal(this.cls, {"x": this.fields["x"] + dx, "y": this.fields["y"] + dy, "color": this.fields["color"]})
    methods = {
        "move": Method(MethSig("move", [("dx", Int), ("dy", Int)], TyClass("ColoredPoint")), move_body),
        "set_color": Method(MethSig("set_color", [("c", Int)], TyClass("ColoredPoint")), set_color_body),
        "is_origin": Method(MethSig("is_origin", [], Bool), lambda this, _: (this.fields["x"] == 0) and (this.fields["y"] == 0))
    }
    return ClassDef("ColoredPoint", "Point", [("x", Int), ("y", Int), ("color", Int)], methods)

CT: ClassTable = {"Point": mk_point_class(), "ColoredPoint": mk_colored_point_class()}

# Sanity check for dispatch and override
p = new_object(CT, "Point", 1, 2)
p2 = call_method(CT, p, "move", [10, -2])            # Point.move
print("ok  - move Point:", p2.fields)

cp = new_object(CT, "ColoredPoint", 1, 2, 7)
cp2 = call_method(CT, cp, "move", [1, 1])            # ColoredPoint.move (override)
print("ok  - move ColoredPoint:", cp2.fields)
print("ok  - is_origin:", call_method(CT, new_object(CT, "Point", 0, 0), "is_origin", []))

In [ ]:
# =====================================
# Part 5 — Interfaces: structural method sets and satisfaction
# =====================================
@dataclass(frozen=True)
class Interface:
    name: str
    methods: List[MethSig]  # required methods (names, arity, param/ret types nominally)

def class_satisfies(CT: ClassTable, cname: str, iface: Interface) -> bool:
    """
    Check whether class 'cname' provides at least the required methods
    (by name and arity; for simplicity we also check printed types).
    """
    try:
        for req in iface.methods:
            m = lookup_method(CT, cname, req.name)
            # compare arity and superficial types
            if len(m.sig.params) != len(req.params):
                return False
            for ((_, ty_req), (_, ty_impl)) in zip(req.params, m.sig.params):
                if show_ty(ty_req) != show_ty(ty_impl):
                    return False
            if show_ty(req.ret) != show_ty(m.sig.ret):
                return False
        return True
    except KeyError:
        return False

# Example interface and checks
Movable = Interface("Movable", [MethSig("move", [("dx", Int), ("dy", Int)], TyClass("Point"))])
print("ok  - Point satisfies Movable? ", class_satisfies(CT, "Point", Movable))
print("ok  - ColoredPoint satisfies Movable? ", class_satisfies(CT, "ColoredPoint", Movable))

In [ ]:
# =====================================
# Part 6 — LSP pitfall: Rectangle/Square and a fix by composition
# =====================================
# "Bad" hierarchy (classic LSP violation)
def mk_rectangle_class() -> ClassDef:
    def set_width(this: ObjVal, args: List[Value]) -> Value:
        (w,) = args
        return ObjVal(this.cls, {"w": w, "h": this.fields["h"]})
    def set_height(this: ObjVal, args: List[Value]) -> Value:
        (h,) = args
        return ObjVal(this.cls, {"w": this.fields["w"], "h": h})
    def area(this: ObjVal, _args: List[Value]) -> Value:
        return this.fields["w"] * this.fields["h"]
    methods = {
        "set_width":  Method(MethSig("set_width",  [("w", Int)], TyClass("Rectangle")), set_width),
        "set_height": Method(MethSig("set_height", [("h", Int)], TyClass("Rectangle")), set_height),
        "area":       Method(MethSig("area", [], TyInt()), area)
    }
    return ClassDef("Rectangle", None, [("w", Int), ("h", Int)], methods)

def mk_square_bad_class() -> ClassDef:
    # Square extends Rectangle but overrides setters to keep sides equal (violates LSP expectation)
    def set_width(this: ObjVal, args: List[Value]) -> Value:
        (w,) = args
        return ObjVal(this.cls, {"w": w, "h": w})
    def set_height(this: ObjVal, args: List[Value]) -> Value:
        (h,) = args
        return ObjVal(this.cls, {"w": h, "h": h})
    def area(this: ObjVal, _args: List[Value]) -> Value:
        return this.fields["w"] * this.fields["h"]
    methods = {
        "set_width":  Method(MethSig("set_width",  [("w", Int)], TyClass("Square")), set_width),
        "set_height": Method(MethSig("set_height", [("h", Int)], TyClass("Square")), set_height),
        "area":       Method(MethSig("area", [], TyInt()), area)
    }
    return ClassDef("Square", "Rectangle", [("w", Int), ("h", Int)], methods)

CT_LSP: ClassTable = {"Rectangle": mk_rectangle_class(), "Square": mk_square_bad_class()}

def client_expect_rectangle(CT: ClassTable, r: ObjVal) -> int:
    # Client expects independent setters: set_width then set_height should give area = w*h
    r1 = call_method(CT, r, "set_width", [3])
    r2 = call_method(CT, r1, "set_height", [4])
    return call_method(CT, r2, "area", [])

print("ok  - LSP: Rectangle client on Rectangle =>", client_expect_rectangle(CT_LSP, new_object(CT_LSP, "Rectangle", 1, 1)))
print("NOTE - LSP: Rectangle client on Square (bad) =>", client_expect_rectangle(CT_LSP, new_object(CT_LSP, "Square", 1, 1)))

# A better design: composition, not inheritance. Square has a single 'size' and supports a common 'area' interface.
def mk_square_good_class() -> ClassDef:
    def set_size(this: ObjVal, args: List[Value]) -> Value:
        (s,) = args
        return ObjVal(this.cls, {"size": s})
    def area(this: ObjVal, _args: List[Value]) -> Value:
        return this.fields["size"] * this.fields["size"]
    methods = {
        "set_size": Method(MethSig("set_size", [("s", Int)], TyClass("SquareGood")), set_size),
        "area":     Method(MethSig("area", [], TyInt()), area)
    }
    return ClassDef("SquareGood", None, [("size", Int)], methods)

CT_fix: ClassTable = {"Rectangle": mk_rectangle_class(), "SquareGood": mk_square_good_class()}

# Common interface Shape with 'area'
Shape = Interface("Shape", [MethSig("area", [], TyInt())])
print("ok  - Rectangle satisfies Shape? ", class_satisfies(CT_fix, "Rectangle", Shape))
print("ok  - SquareGood satisfies Shape? ", class_satisfies(CT_fix, "SquareGood", Shape))

### Your Turn (10–15 minutes)
1. Extend `Interface`/`class_satisfies` to also check **method names** and **parameter names** (not only arity/types). Show one failing example.
2. Add a **trait/mixin** as a dict of methods (e.g., `Loggable` with `log(msg:Int): Int`) and create a new class by copying a base class and merging methods. Show dynamic dispatch still works.
3. Write a short paragraph explaining why the original `Square extends Rectangle` breaks **LSP**, and how the composition design restores substitutability.

### Reflection (2–3 sentences)
- In your words, how does **dynamic dispatch** differ from a static function table?  
- Why is **interface‑based programming** often safer than subclassing for behavioral contracts?

In [ ]:
# Save small submission bundle
import json, time
stamp = time.strftime("%Y-%m-%d %H:%M:%S")
submission = {
  "student_name": STUDENT_NAME,
  "student_id": STUDENT_ID,
  "timestamp": stamp,
  "checks": ["dispatch", "interfaces", "lsp-demo"],
  "reflection": "(fill in here)"
}
with open("week7_submission.json", "w") as f:
  json.dump(submission, f, indent=2)
print("Saved week7_submission.json — upload with your notebook.")